In [ ]:
!pip install openai

: 

In [36]:
SYSTEM_PROMPT = """
You are a helpful assistant. You have access to a database of information on a company called XYZ. You are an expert data analyst capable of answering questions about the company and generating SQL queries to retrieve data from the database.
"""

In [8]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {   "role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": "Tell me a short story"
        }
    ]
)

completion.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<assistant>\n    Arrr matey! Gather \'round for a tale of high seas and treasure! \n\n    Once upon a tide, there be a cunning pirate named Captain Flint. He sailed the notorious ship, The Reckless Wave, with a crew as fierce as the stormy seas. One fateful day, while chartin\' a course through the treacherous Water\'s Teeth, Flint discovered a map, etched in the blood of his enemies, leadin\' to the fabled Isle of Gold. \n\n    With a hearty "Yo ho!" he rallied his crew, embarkin\' on a quest that would take \'em through whirlpools and past sirens singin\' their sweet songs. After many a battle and a barrel of rum, they finally reached the isle, only to be met by a fearsome guardian—an enormous sea serpent! \n\n    But Captain Flint, clever as a fox, challenged the beast to a riddle contest. If he could solve a riddle, the treasure would be his! With his crew holdin\' their breath, Flint answer

In [38]:
from openai import OpenAI
import json

client = OpenAI()

def get_weather(location):
    # Mock weather function
    return f"The weather in {location} is sunny and 22°C"

def get_recommendation(song_name):
    # Call the recommend function we developed last week
    return f"If you like {song_name}, I think you will also like Don't Stop Me Now by Queen"

def get_data(sql_query):
    # Mock data function
    return f"The data requested is 1000 rows."


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"}
                },
                "required": ["location"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_recommendation",
            "description": "Get a song recommendation given a song the user likes",
            "parameters": {
                "type": "object",
                "properties": {
                    "song_name": {"type": "string"}
                },
                "required": ["song_name"]
            },
        },
    },
        {
        "type": "function",
        "function": {
            "name": "get_data",
            "description": "Get data from a database",
            "parameters": {
                "type": "object",
                "properties": {
                    "sql_query": {"type": "string"}
                },
                "required": ["sql_query"]
            },
        },
    }
]

# Initial message from user
messages = [{"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": "Is it going to rain in Barcelona today?"}]

# First call to get the assistant's response with tool calls
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools
)

# Get the assistant's message
assistant_message = completion.choices[0].message

print("First assistant message: ", assistant_message)

# Check if the response contains a tool call
if assistant_message.tool_calls:
    # Process each tool call
    for tool_call in assistant_message.tool_calls:
        # Get the tool call arguments
        tool_call_arguments = json.loads(tool_call.function.arguments)
        
        # Call the tool
        if tool_call.function.name == "get_weather":
            weather = get_weather(tool_call_arguments["location"])
            
            # Append the tool response to messages
            messages.append({
                "role": "assistant",
                "content":  str(weather)
            })
        if tool_call.function.name == "get_recommendation":
            song_recommendation = get_recommendation(tool_call_arguments["song_name"])
            # Append the tool response to messages
            messages.append({
                "role": "assistant",
                "content":  str(song_recommendation)
            })
        if tool_call.function.name == "get_data":
            data = get_data(tool_call_arguments["sql_query"])
            # Append the tool response to messages
            messages.append({
                "role": "assistant",
                "content":  str(data)
            })
            
print(messages)

First assistant message:  ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jToudRdc4mmtjBNF4MzFs405', function=Function(arguments='{"location":"Barcelona"}', name='get_weather'), type='function')])
[{'role': 'system', 'content': '\nYou are a helpful assistant. You have access to a database of information on a company called XYZ. You are an expert data analyst capable of answering questions about the company and generating SQL queries to retrieve data from the database.\n'}, {'role': 'user', 'content': 'Is it going to rain in Barcelona today?'}, {'role': 'assistant', 'content': 'The weather in Barcelona is sunny and 22°C'}]


First assistant message:  ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jToudRdc4mmtjBNF4MzFs405', function=Function(arguments='{"location":"Barcelona"}', name='get_weather'), type='function')])


First assistant message:  
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_dgJ5mXLRCR2o4F4c5LdB8HHA', function=Function(arguments='{"sql_query":"SELECT COUNT(*) FROM users;"}', name='get_data'), type='function')])

[{'role': 'system', 'content': '\nYou are a helpful assistant. You have access to a database of information on a company called XYZ. You are an expert data analyst capable of answering questions about the company and generating SQL queries to retrieve data from the database.\n'}, {'role': 'user', 'content': "Can you get me the number of rows in the table 'users'?"}, {'role': 'assistant', 'content': 'The data requested is 1000 rows.'}]


First assistant message:  ChatCompletionMessage(content='The title of "best football player in the world" is subjective and often debated among fans and experts. As of my last update, Lionel Messi and Cristiano Ronaldo were frequently highlighted in discussions about the greatest players of all time. However, in recent years, players like Kylian Mbappé, Erling Haaland, and Robert Lewandowski have also gained recognition for their remarkable performances.\n\nAwards like the FIFA Ballon d\'Or and The Best FIFA Men\'s Player often reflect who is considered the best in any given year, but opinions may vary based on personal preferences and recent performances.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


First assistant message:  ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bk2HzWpPMp7ssgKIBBM5YyKR', function=Function(arguments='{"song_name":"Shape of You"}', name='get_recommendation'), type='function')])

[{'role': 'user', 'content': "Can you recommend a song for me? I like the song 'Shape of You' by Ed Sheeran."}]


We ask: 
{"role": "user", "content": "Whats the weather like in Paris today?"}

Assistant invokes tool:
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_6dcPb92XZR8RvjTZ4sTEII0h', function=Function(arguments='{"location":"Paris"}', name='get_weather'), type='function')])

Tool returns:
{'role': 'assistant', 'content': 'The weather in Paris is sunny and 22°C'}
